In [31]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

In [34]:

# DATA GENERATION CODE
# def process_md_files(folder_path, ignore_folder):
#     markdownFiles = []
#     for root, dirs, files in os.walk(folder_path):
#         if ignore_folder in dirs:
#             dirs.remove(ignore_folder)
#         for file in files:
#             if file.endswith(".md"):
#                 file_path = os.path.join(root, file)
#                 markdownFiles.append(UnstructuredMarkdownLoader(file_path))
#     return markdownFiles

# folder_path = "./input"
# ignore_folder = "output"
# loaders = process_md_files(folder_path, ignore_folder)

# docs = []
# for file in loaders:
#     docs.extend(file.load())
    
# #split text to chunks
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# docs = text_splitter.split_documents(docs)
# embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': 'cpu'})

# vectorstore = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_db_nccn")

# print(vectorstore._collection.count())

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [33]:
client = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

embedding_function=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = Chroma(persist_directory="./chroma_db_nccn", embedding_function=embedding_function)

retriever = vector_db.as_retriever(search_kwargs={"k": 1})

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def print_and_format_docs(docs):
    for doc in docs:
        print(f"Document: {doc}")
    return format_docs(docs)

rag_chain = (
    {"context": retriever | print_and_format_docs, "question": RunnablePassthrough()}
    | prompt
    | client
    | StrOutputParser()
)

rag_chain.invoke("What is a good Greek word to use when translating the word 'Warehouse'")

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Document: page_content='ἀποθήκη/apothēkē\n\nThis word is a type of place. It is a noun.\n\nThis word means a storehouse or barn. This is a building where grain is stored.' metadata={'source': './input\\en_gwt\\g0591-g0600\\g0596.md'}


" αποθηκη (apothēkē) can be used to translate 'Warehouse' as it denotes a place for storage, specifically referring to a storehouse or a space storing grains."